In [1]:
import pandas as pd
from torchvision.models.efficientnet import efficientnet_v2_l, EfficientNet_V2_L_Weights
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from sklearn.model_selection import train_test_split


from classification import ClassificationHead
from dataset import SportacusDataset

# Datasets

In [2]:
data_path = Path("./../data")
train_folder = data_path / "train"
test_folder = data_path / "test"
train_csv = data_path / "train.csv"
test_csv = data_path / "test.csv"

In [3]:
transforms = EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms()

df = pd.read_csv(train_csv)
train_ind, val_ind = train_test_split(df.index, test_size=30, shuffle=False)

train_dataset = SportacusDataset(df.iloc[train_ind, :], train_folder, min_shape_size=40, transform=transforms)
val_dataset = SportacusDataset(df.iloc[val_ind, :], train_folder, min_shape_size=40, transform=transforms)


In [4]:
# BATCH_SIZE = 32
# train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, drop_last=True)
# val_dataloader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False, drop_last=True)

# Model creation

In [4]:
sportacus = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.DEFAULT)
for param in sportacus.parameters():
    param.requires_grad = False

classification_head = ClassificationHead(30)
sportacus.classifier = classification_head

In [1]:
# from torchvision.models.efficientnet import efficientnet_v2_m, EfficientNet_V2_M_Weights
# model = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)

from torchvision.models.vision_transformer import (
    vit_b_32,
    ViT_B_32_Weights,
)

model = vit_b_32(weights=ViT_B_32_Weights.IMAGENET1K_V1)
model

Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to /root/.cache/torch/hub/checkpoints/vit_b_32-d86f8d99.pth
100%|██████████| 337M/337M [00:08<00:00, 43.2MB/s] 


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

# Train

In [5]:
import lightning.pytorch as pl
from torch.nn import Module, NLLLoss
from torch.nn.functional import softmax
from torch import Tensor, optim, set_float32_matmul_precision
from typing import Tuple, Optional, Callable
from sklearn.metrics import f1_score


def lr(step):
    return 0.1 + 0.9 * (0.998) ** step


def make_lr_scheduler(optimizer):
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr)


def make_optimizer(model):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
    return optimizer


class SportacusModule(pl.LightningModule):
    def __init__(
        self,
        model: Module,
        optimizer_fn: Callable[[Module], optim.Optimizer],
        trainset: SportacusDataset,
        testset: SportacusDataset,
        lr_scheduler_fn: Optional[optim.lr_scheduler.LRScheduler] = None,
        batch_size=32,
    ):
        super().__init__()
        self._model = model
        self._optimizer_fn = optimizer_fn
        self._lr_scheduler_fn = lr_scheduler_fn
        self._criterion = NLLLoss(trainset.class_w)
        self._batch_size = batch_size
        self._trainset = trainset
        self._testset = testset

    def forward(self, input):
        return self._model(input)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self._model(x)
        loss = self._criterion(logits, y)
        self.logger.experiment.add_scalars(
            "loss",
            {"train": loss},
            global_step=self.global_step,
        )
        self.logger.experiment.add_scalars(
            "f1",
            {"train": self._f1(logits, y)},
            global_step=self.global_step,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self._model(x)
        loss = self._criterion(logits, y)
        self.logger.experiment.add_scalars(
            "loss", {"val": loss}, global_step=self.global_step,
        )
        self.logger.experiment.add_scalars(
            "f1",
            {"val": self._f1(logits, y)},
            global_step=self.global_step,
        )

    def train_dataloader(self):
        return DataLoader(
            self._trainset, batch_size=self._batch_size, shuffle=True, drop_last=True
        )

    def val_dataloader(self):
        return DataLoader(self._testset, batch_size=self._batch_size, shuffle=False, num_workers=16)

    def configure_optimizers(self):
        optimizer = self._optimizer_fn(self._model)
        scheduler = self._lr_scheduler_fn(optimizer)
        return [optimizer], [scheduler]

    def _f1(self, logits, labels):
        np_target = labels.cpu().detach().numpy()
        np_logits = softmax(logits).cpu().detach().numpy()
        return f1_score(np_target, np_logits, average="weighted")


In [6]:
set_float32_matmul_precision('high')
model = SportacusModule(
    sportacus,
    make_optimizer,
    train_dataset,
    val_dataset,
    make_lr_scheduler,
    32,
)
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    val_check_interval=250,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model)

Missing logger folder: /workspaces/cv/src/notebooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | _model     | EfficientNet | 119 M 
1 | _criterion | NLLLoss      | 0     
--------------------------------------------
2.5 M     Trainable params
117 M     Non-trainable params
119 M     Total params
478.988   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

: 

: 

In [3]:
from ..pltrain import init_model

ImportError: attempted relative import with no known parent package